In [1]:
# ChemGraph Day 0 - Google Colab Version

# Install libraries
!pip install neo4j google-generativeai fuzzywuzzy python-Levenshtein requests python-dotenv -q

# Create environment file
import os
from google.colab import userdata



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.8 MB/s eta 0:00:00


In [2]:
# Helper functions for all 3 tests

from google.colab import userdata
import os
from neo4j import GraphDatabase
import google.generativeai as genai
from fuzzywuzzy import fuzz
import json
import re

# Load secrets from Colab
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USER = userdata.get('NEO4J_USER')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

print(f"✅ Loaded secrets from Colab")
print(f"   Neo4j URI: {NEO4J_URI[:30]}...")
print(f"   Gemini API: Configured")


✅ Loaded secrets from Colab
   Neo4j URI: neo4j+s://5ea1496f.databases.n...
   Gemini API: Configured


In [7]:
# TEST 1: Gemini can extract chemical interactions from papers

print("=" * 60)
print("TEST 1: Gemini Extraction from Toxicology Paper")
print("=" * 60)

def test_gemini_extraction():
    """
    We'll test Gemini with a hardcoded toxicology example
    (Instead of uploading a PDF, which requires manual download)
    """

    model = genai.GenerativeModel("gemini-2.5-flash")

    # Example paper excerpt (real toxicology text)
    paper_excerpt = """
    RESULTS: We investigated the interaction between sodium benzoate and ascorbic acid.
    When sodium benzoate (100 mg/L) was combined with ascorbic acid (50 mg/L) in aqueous
    solution at pH 3.5, we observed formation of benzene as a byproduct. Gas chromatography
    confirmed the presence of benzene at 2.3 ppb after 24 hours. The interaction was
    dose-dependent, with higher concentrations producing more benzene. Similar interactions
    were observed with other preservatives. CONCLUSION: The combination of sodium benzoate
    and vitamin C should be avoided in acidic food products due to potential carcinogen formation.
    Confidence in this finding: High (based on 5 replicate experiments).

    METHODS: We also studied titanium dioxide nanoparticles (E171). In simulated gastric
    conditions, nanoparticles accumulated in intestinal epithelial cells with 87% retention.
    This interaction with the gut barrier poses potential genotoxicity concerns.
    Confidence: Medium (animal study, n=20 mice).
    """

    prompt = """
    Extract all chemical interactions from this toxicology paper excerpt.
    For each interaction, provide:
    1. Chemical A (exact name)
    2. Chemical B (exact name)
    3. Relationship (INTERACTS_WITH / ENHANCES / CREATES_TOXIC_COMPOUND)
    4. Resultant compound (what is created)
    5. Health impact (Carcinogenic / Mutagenic / Genotoxic / etc)
    6. Confidence (0-1, based on study quality)

    Return ONLY valid JSON array - NO OTHER TEXT:
    [
        {
            "chemical_a": "...",
            "chemical_b": "...",
            "relationship": "...",
            "resultant": "...",
            "health_impact": "...",
            "confidence": 0.85
        }
    ]

    Paper excerpt:
    """ + paper_excerpt

    try:
        response = model.generate_content(prompt)

        # Extract JSON from response
        json_match = re.search(r'\[.*\]', response.text, re.DOTALL)
        if json_match:
            triplets = json.loads(json_match.group())
            return True, triplets
        else:
            return False, f"No JSON found in response: {response.text[:200]}"
    except Exception as e:
        return False, str(e)

# Run test
success, result = test_gemini_extraction()

if success:
    print(f"\n✅ TEST 1 PASSED!")
    print(f"   Extracted {len(result)} interactions")
    print("\n📋 Interactions found:")
    for i, triplet in enumerate(result, 1):
        print(f"\n   {i}. {triplet['chemical_a']} + {triplet['chemical_b']}")
        print(f"      → {triplet['resultant']} ({triplet['health_impact']})")
        print(f"      Confidence: {triplet['confidence']:.0%}")
else:
    print(f"\n❌ TEST 1 FAILED: {result}")


TEST 1: Gemini Extraction from Toxicology Paper

✅ TEST 1 PASSED!
   Extracted 2 interactions

📋 Interactions found:

   1. sodium benzoate + ascorbic acid
      → benzene (Carcinogenic)
      Confidence: 90%

   2. titanium dioxide nanoparticles (E171) + gut barrier
      →  (Genotoxic)
      Confidence: 60%


In [6]:
import google.generativeai as genai

genai.configure(api_key="your-key-here")

# List all available models
for model in genai.list_models():
    print(f"Name: {model.name}")
    print(f"Supported generation methods: {model.supported_generation_methods}")
    print("-" * 40)

# Then use one of the listed model names

Name: models/embedding-gecko-001
Supported generation methods: ['embedText', 'countTextTokens']
----------------------------------------
Name: models/gemini-2.5-flash
Supported generation methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
----------------------------------------
Name: models/gemini-2.5-pro
Supported generation methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
----------------------------------------
Name: models/gemini-2.0-flash-exp
Supported generation methods: ['generateContent', 'countTokens', 'bidiGenerateContent']
----------------------------------------
Name: models/gemini-2.0-flash
Supported generation methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
----------------------------------------
Name: models/gemini-2.0-flash-001
Supported generation methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
-------

In [8]:
# TEST 2: Fuzzy matching works without LLM

print("\n" + "=" * 60)
print("TEST 2: Fuzzy Matching (OCR Typo Correction)")
print("=" * 60)

def load_ingredient_dictionary():
    """Common food ingredients"""
    return [
        "Sodium Benzoate",
        "Ascorbic Acid",
        "Tartrazine E102",
        "Titanium Dioxide E171",
        "High Fructose Corn Syrup",
        "Sodium Nitrate",
        "Potassium Sorbate",
        "Citric Acid",
        "Sugar",
        "Salt",
        "Water",
        "Carrageenan",
        "Xanthan Gum",
        "Monosodium Glutamate",
        "Aspartame",
        "Sucralose",
    ]

def fuzzy_match(ocr_text, ingredient_dict):
    """Match OCR output to known ingredient"""
    best_match = None
    best_score = 0

    for ingredient in ingredient_dict:
        score = fuzz.token_set_ratio(
            ocr_text.lower(),
            ingredient.lower()
        )
        if score > best_score:
            best_score = score
            best_match = ingredient

    if best_score >= 80:
        return {
            "ingredient": best_match,
            "confidence": best_score / 100.0
        }
    return None

def test_fuzzy_matching():
    """Test fuzzy matcher on realistic OCR outputs"""

    ingredient_dict = load_ingredient_dictionary()

    test_cases = [
        ("Sodum Benzoate", "Sodium Benzoate", True),        # OCR typo
        ("Ascorbic Acid", "Ascorbic Acid", True),           # Perfect
        ("E102", "Tartrazine E102", True),                  # E-code
        ("Titanium Dioxide", "Titanium Dioxide E171", True),# Partial
        ("Sugar", "Sugar", True),                           # Common
        ("Xyz Random", None, False),                        # Unknown
    ]

    passed = 0
    failed = 0

    print("\n📝 Running 6 test cases:\n")

    for ocr_text, expected, should_pass in test_cases:
        result = fuzzy_match(ocr_text, ingredient_dict)

        if should_pass:
            if result and result["ingredient"] == expected:
                print(f"✅ '{ocr_text}' → '{result['ingredient']}' ({result['confidence']:.0%})")
                passed += 1
            else:
                print(f"❌ '{ocr_text}' failed (expected '{expected}')")
                failed += 1
        else:
            if result is None:
                print(f"✅ '{ocr_text}' correctly rejected")
                passed += 1
            else:
                print(f"❌ '{ocr_text}' should have been rejected")
                failed += 1

    return passed, failed

passed, failed = test_fuzzy_matching()

print(f"\n{'='*60}")
if failed == 0:
    print(f"✅ TEST 2 PASSED! ({passed}/{passed+failed} cases correct)")
else:
    print(f"❌ TEST 2 FAILED ({failed} cases failed)")



TEST 2: Fuzzy Matching (OCR Typo Correction)

📝 Running 6 test cases:

✅ 'Sodum Benzoate' → 'Sodium Benzoate' (97%)
✅ 'Ascorbic Acid' → 'Ascorbic Acid' (100%)
✅ 'E102' → 'Tartrazine E102' (100%)
✅ 'Titanium Dioxide' → 'Titanium Dioxide E171' (100%)
✅ 'Sugar' → 'Sugar' (100%)
✅ 'Xyz Random' correctly rejected

✅ TEST 2 PASSED! (6/6 cases correct)


In [9]:
# TEST 3: Neo4j digital twin model works

print("\n" + "=" * 60)
print("TEST 3: Neo4j Digital Twin Query")
print("=" * 60)

def test_neo4j_connection():
    """Test Neo4j connection and digital twin model"""

    try:
        driver = GraphDatabase.driver(
            NEO4J_URI,
            auth=(NEO4J_USER, NEO4J_PASSWORD)
        )

        with driver.session() as session:
            # Test 1: Connection
            result = session.run("RETURN 'Connected!' as message")
            message = result.single()["message"]
            print(f"\n✅ Connected to Neo4j: {message}")

            # Test 2: Create digital twin
            print("\n   Creating digital twin node...")
            session.run("""
                MERGE (u:User {id: 'test_user_001'})
                SET u.age = 28, u.weight_kg = 70, u.conditions = ['Diabetes']
            """)
            print("   ✅ User node created")

            # Test 3: Create products
            print("\n   Creating product nodes...")
            session.run("""
                MERGE (p1:Product {name: 'Heinz Classic Ketchup'})
                SET p1.category = 'Condiment', p1.risk_score = 0.65
            """)
            session.run("""
                MERGE (p2:Product {name: 'Heinz Organic Ketchup'})
                SET p2.category = 'Condiment', p2.risk_score = 0.25
            """)
            session.run("""
                MERGE (p3:Product {name: 'Homemade Tomato Sauce'})
                SET p3.category = 'Condiment', p3.risk_score = 0.05
            """)
            print("   ✅ Product nodes created")

            # Test 4: Link user consumed product
            print("\n   Creating consumption relationship...")
            session.run("""
                MATCH (u:User {id: 'test_user_001'})
                MATCH (p:Product {name: 'Heinz Classic Ketchup'})
                MERGE (u)-[:CONSUMED {timestamp: '2025-12-06'}]->(p)
            """)
            print("   ✅ Consumption link created")

            # Test 5: Query for alternatives
            print("\n   Querying for healthier alternatives...")
            result = session.run("""
                MATCH (u:User {id: 'test_user_001'})
                MATCH (rejected:Product {name: 'Heinz Classic Ketchup'})
                MATCH (alternative:Product)
                WHERE alternative.category = rejected.category
                AND alternative.risk_score < rejected.risk_score
                RETURN alternative.name, alternative.risk_score
                ORDER BY alternative.risk_score ASC
                LIMIT 5
            """)

            alternatives = result.data()

            if alternatives:
                print(f"\n✅ TEST 3 PASSED!")
                print(f"   Found {len(alternatives)} healthier alternatives:")
                for alt in alternatives:
                    print(f"   • {alt['alternative.name']} (risk: {alt['alternative.risk_score']:.2f})")
                return True
            else:
                print("\n❌ TEST 3 FAILED: No alternatives found")
                return False

        driver.close()

    except Exception as e:
        print(f"\n❌ TEST 3 FAILED: {str(e)}")
        print("\nTroubleshooting:")
        print("1. Check Neo4j URI in secrets (should start with neo4j+s://)")
        print("2. Verify username/password are correct")
        print("3. Wait 5 minutes for AuraDB to fully deploy")
        return False

success = test_neo4j_connection()



TEST 3: Neo4j Digital Twin Query

✅ Connected to Neo4j: Connected!

   Creating digital twin node...
   ✅ User node created

   Creating product nodes...
   ✅ Product nodes created

   Creating consumption relationship...
   ✅ Consumption link created

   Querying for healthier alternatives...

✅ TEST 3 PASSED!
   Found 2 healthier alternatives:
   • Homemade Tomato Sauce (risk: 0.05)
   • Heinz Organic Ketchup (risk: 0.25)
